<a href="https://colab.research.google.com/github/FARDIN98/AI-lab/blob/main/explanation_of_simulated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The N-Queen problem is a classic problem in which N queens are to be placed on an NxN chessboard such that no two queens threaten each other (i.e., no two queens share the same row, column, or diagonal).

Here's a broad explanation of the code:

The Queen class is defined to represent the chessboard and the placement of queens. It has methods to add queens randomly to the board and get neighboring positions of a cell.

The conflict function is used to check if there's a conflict between two queens in terms of their row, column, or diagonal positions.

The get_conflict function calculates the number of conflicts a queen has with other queens on the chessboard.

The calc_cost function computes the total cost of conflicts and identifies the queen with the maximum conflicts.

The goal_test function checks if the current state is the goal state (i.e., no conflicts between queens).

The move_queen function randomly moves a queen to a neighboring cell on the chessboard.

The simulated_annealing function performs the Simulated Annealing algorithm to find a solution for the N-Queen problem. It starts with an initial temperature and iteratively decreases the temperature to explore the solution space.

The queen object is created with a 4x4 chessboard and initial queen placements.

The simulated_annealing function is called to find a solution by iteratively moving queens to neighboring positions while considering the temperature and cost difference.

Finally, the final state of the queens and the maximum number of conflicts are printed.

In summary, the code uses Simulated Annealing to find a solution to the N-Queen problem by iteratively exploring different configurations of queens on the chessboard and accepting moves that reduce conflicts while considering the temperature. The algorithm is probabilistic, and it aims to find a near-optimal solution to the problem.**bold text**

In [1]:
import numpy as np  # Import the NumPy library for array operations

class Queen:
    def __init__(self, N):
        # Initialize the Queen class with a given board size N and create data structures to store queens' locations
        self.N = N
        self.queen_loc = dict()  # Dictionary to store the locations of queens
        self.initialize = False  # Flag to indicate if the queens are initialized
        self.chess_board = [[0]*self.N for _ in range(self.N)]  # 2D list representing the chessboard

    def add_queen(self):
        # Method to add queens randomly to the chessboard
        if self.initialize == False:
            number_Q = 0  # Counter to keep track of the number of queens placed
            while True:
                flag = 0  # Flag to check if the position is already occupied by another queen
                r = np.random.randint(self.N)  # Generate a random row index
                c = np.random.randint(self.N)  # Generate a random column index
                for q in self.queen_loc:
                    row, col = self.queen_loc[q]
                    if (r == row and c == col) or (c == col):
                        flag = 1  # Set the flag to 1 if there's a conflict in row or column
                if flag == 0:  # If no conflict found, place the queen at (r, c)
                    Q = f"Q{number_Q}"  # Generate a unique identifier for the queen
                    if Q not in self.queen_loc:
                        self.queen_loc[Q] = []  # Initialize the queen's location list
                    self.queen_loc[Q].append(r)  # Store the row index for the queen
                    self.queen_loc[Q].append(c)  # Store the column index for the queen
                    self.chess_board[r][c] = Q  # Update the chessboard with the queen's identifier
                    number_Q += 1  # Increment the number of placed queens
                if number_Q == self.N:  # If all queens are placed, break out of the loop
                    break
            self.initialize = True  # Set the initialize flag to True after placing all queens

    def get_neighbor(self, row, col):
        # Get neighboring positions of a given cell on the chessboard
        neighbor = []
        if 0 <= row-1 < self.N and self.chess_board[row-1][col] == 0:
            neighbor.append([row-1, col])  # Check the cell above if it's empty
        if 0 <= row+1 < self.N and self.chess_board[row+1][col] == 0:
            neighbor.append([row+1, col])  # Check the cell below if it's empty
        return neighbor  # Return the list of neighboring positions

    def print_Queen(self):
        # Print the chessboard and the locations of queens
        print(self.chess_board)
        for Q in self.queen_loc:
            print(f'{Q}->{self.queen_loc[Q]}')  # Print the location of each queen

def conflict(r1, c1, r2, c2):
    # Check if there's a conflict between two queens in the same row, column, or diagonal
    if r1 == r2:
        return True
    if c1 == c2:
        return True
    if r1 + c1 == r2 + c2:
        return True
    if r1 - c1 == r2 - c2:
        return True
    return False

def get_conflict(Q, state):
    # Calculate the number of conflicts a queen has with other queens on the chessboard
    count = 0
    for q in state:
        if q is not Q:
            r1, c1 = state[Q]
            r2, c2 = state[q]
            if conflict(r1, c1, r2, c2):
                count += 1
    return count

def calc_cost(state):
    # Calculate the total cost of conflicts and the queen with the maximum conflicts
    cost = 0
    max_conflicts = -999
    maxQ = None
    for Q in state:
        q_cost = get_conflict(Q, state)
        cost += q_cost
        if q_cost > max_conflicts:
            max_conflicts = q_cost
            maxQ = Q
    return cost // 2, max_conflicts, maxQ  # Return the total cost, maximum conflicts, and the queen identifier with maximum conflicts

def goal_test(state):
    # Check if the current state is the goal state (no conflicts between queens)
    if calc_cost(state)[0] == 0:
        return True
    else:
        return False

def move_queen(queen):
    # Move a queen to a neighboring cell on the chessboard
    while True:
        selected_queen = np.random.choice(list(queen.queen_loc.keys()))  # Choose a random queen
        current_row, current_col = queen.queen_loc[selected_queen]  # Get the current location of the selected queen

        neighbors = queen.get_neighbor(current_row, current_col)  # Get the neighbors of the selected queen
        if not neighbors:
            return False  # If there are no valid neighbors, return False to indicate no move was made

        neighbors = np.array(neighbors)  # Convert neighbors list to a NumPy array
        new_row, new_col = np.random.choice(neighbors[:, 0]), np.random.choice(neighbors[:, 1])  # Choose a random valid neighbor
        queen.chess_board[current_row][current_col] = 0  # Clear the current cell of the selected queen
        queen.chess_board[new_row][new_col] = selected_queen  # Move the selected queen to the new cell
        queen.queen_loc[selected_queen] = [new_row, new_col]  # Update the location of the selected queen in queen_loc dictionary
        return True  # Return True to indicate a successful move

def simulated_annealing(queen, initial_temperature=1000, cooling_rate=0.99, stopping_temperature=0.001):
    # Simulated Annealing algorithm to solve the N-Queen problem
    current_state = queen.queen_loc.copy()  # Make a copy of the current state of the queens
    current_cost, max_cost, _ = calc_cost(current_state)  # Calculate the cost of the current state

    temperature = initial_temperature  # Set the initial temperature
    while temperature > stopping_temperature:  # Annealing loop until stopping temperature is reached
        move_queen(queen)  # Move a queen to a neighboring position

        new_state = queen.queen_loc.copy()  # Make a copy of the new state after the move
        new_cost, new_max_cost, _ = calc_cost(new_state)  # Calculate the cost of the new state

        cost_diff = new_cost - current_cost  # Calculate the difference in costs

        if cost_diff < 0 or np.random.rand() < np.exp(-cost_diff / temperature):
            # If the new state has a lower cost or with a certain probability, accept the move
            current_state = new_state  # Update the current state to the new state
            current_cost = new_cost  # Update the current cost to the new cost
            max_cost = new_max_cost  # Update the maximum cost
        else:
            # Otherwise, reject the move and revert the queen to the previous state
            queen.queen_loc = current_state
            queen.chess_board = [[0] * queen.N for _ in range(queen.N)]
            for Q in current_state:
                row, col = current_state[Q]
                queen.chess_board[row][col] = Q

        temperature *= cooling_rate  # Reduce the temperature in the annealing process

    return max_cost, current_state  # Return the maximum conflicts and the final state after annealing

queen = Queen(4)  # Create a 4x4 chessboard with queens
queen.queen_loc = {'Q0': [1, 1], 'Q1': [1, 3], 'Q2': [2, 0], 'Q3': [2, 2]}  # Set the initial state of queens
queen.chess_board = [['Q0', 0, 'Q1', 0], [0, 0, 0, 0], ['Q2', 0, 'Q3', 0], [0, 0, 0, 0]]  # Set the initial chessboard state
queen.print_Queen()  # Print the initial state of the queens

max_conflicts, final_state = simulated_annealing(queen)  # Perform simulated annealing to find a solution
print("Final State:")
queen.queen_loc = final_state  # Update the queens' locations to the final state
queen.print_Queen()  # Print the final state of the queens
print("Max Conflicts:", max_conflicts)  # Print the maximum number of conflicts in the final state


[['Q0', 0, 'Q1', 0], [0, 0, 0, 0], ['Q2', 0, 'Q3', 0], [0, 0, 0, 0]]
Q0->[1, 1]
Q1->[1, 3]
Q2->[2, 0]
Q3->[2, 2]
Final State:
[[0, 'Q0', 0, 0], [0, 0, 0, 'Q1'], ['Q2', 0, 'Q3', 0], [0, 0, 0, 0]]
Q0->[0, 1]
Q1->[1, 3]
Q2->[2, 0]
Q3->[2, 2]
Max Conflicts: 0
